# Class 2: Time Series Analysis with Statistical Modeling

This notebook delves into statistical models commonly used for time series analysis and forecasting. We will cover AR, MA, ARMA, ARIMA, SARIMAX, and GARCH models, along with model selection criteria and diagnostic checks.

## 0. Setup and Imports

In [ ]:

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import warnings
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
import pmdarima as pm
from arch import arch_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['axes.grid'] = True


## 1. Data Loading and Preparation

In [ ]:

# Download 10 years of Apple stock data
start_date = '2015-01-01'
end_date = '2025-01-01'
ticker = 'AAPL'

# Fetch data using yfinance
df = yf.download(ticker, start=start_date, end=end_date)
print(f"Downloaded {ticker} stock data from {start_date} to {end_date}")
print(f"Shape of data: {df.shape}")

# Display the first few rows
print("\nFirst 5 rows of the data:")
display(df.head())

# We'll focus on the 'Adj Close' price for our analysis
ts = df['Adj Close']
print(f"\nFocusing on Adjusted Close price for {ticker}")

# Create log returns for GARCH modeling
log_returns = np.log(ts / ts.shift(1)).dropna()
print("\nCreated log returns series for GARCH modeling")

# Split data into training and testing sets (80% train, 20% test)
train_size = int(len(ts) * 0.8)
train_ts = ts[:train_size]
test_ts = ts[train_size:]
print(f"\nSplit data into training ({len(train_ts)} samples) and testing ({len(test_ts)} samples) sets")

# Also split the log returns
train_returns = log_returns[:train_size]
test_returns = log_returns[train_size:]

# Plot the training and testing data
plt.figure(figsize=(14, 7))
plt.plot(train_ts, label='Training Data')
plt.plot(test_ts, label='Testing Data')
plt.title(f'{ticker} Stock Price - Train/Test Split')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.tight_layout()
plt.show()


## 2. AR, MA, ARMA, and ARIMA Models

AR (Autoregressive) Model concept not found

MA (Moving Average) Model concept not found

ARIMA Model concept not found

In [ ]:

# Fit ARIMA(1,1,1) model on the training data
arima_model = ARIMA(train_ts, order=(1, 1, 1))
arima_results = arima_model.fit()

# Print model summary
print("ARIMA(1,1,1) Model Summary:")
print(arima_results.summary())

# Forecast on the test set
arima_forecast = arima_results.forecast(steps=len(test_ts))

# Calculate error metrics
arima_rmse = math.sqrt(mean_squared_error(test_ts, arima_forecast))
arima_mae = mean_absolute_error(test_ts, arima_forecast)
print(f"ARIMA RMSE: {arima_rmse:.4f}")
print(f"ARIMA MAE: {arima_mae:.4f}")

# Plot the forecast
plt.figure(figsize=(14, 7))
plt.plot(train_ts, label='Training Data')
plt.plot(test_ts, label='Actual Test Data')
plt.plot(test_ts.index, arima_forecast, label='ARIMA Forecast', color='red')
plt.title('ARIMA(1,1,1) Forecast')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.tight_layout()
plt.savefig('plot_09_arima_forecast.png')
plt.show()


**Interpretation:**


The ARIMA(1,1,1) model combines:
- AR(1): Autoregressive component with lag 1
- I(1): First-order differencing to make the series stationary
- MA(1): Moving average component with lag 1

The model summary shows:
- Coefficients for AR and MA terms
- Statistical significance of each parameter
- AIC, BIC, and other model fit statistics

The forecast performance metrics (RMSE and MAE) provide a baseline for comparison with other models.


## 3. AUTO ARIMA Model

In [ ]:

# Fit Auto ARIMA model
auto_arima_model = pm.auto_arima(
    train_ts,
    start_p=0, start_q=0,
    max_p=5, max_q=5, max_d=2,
    seasonal=False,
    trace=True,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True
)

# Print model summary
print("\nAuto ARIMA Model Summary:")
print(auto_arima_model.summary())

# Get the order of the best model
best_order = auto_arima_model.order
print(f"\nBest ARIMA order: {best_order}")

# Forecast on the test set
auto_arima_forecast = auto_arima_model.predict(n_periods=len(test_ts))

# Calculate error metrics
auto_arima_rmse = math.sqrt(mean_squared_error(test_ts, auto_arima_forecast))
auto_arima_mae = mean_absolute_error(test_ts, auto_arima_forecast)
print(f"Auto ARIMA RMSE: {auto_arima_rmse:.4f}")
print(f"Auto ARIMA MAE: {auto_arima_mae:.4f}")

# Plot the forecast
plt.figure(figsize=(14, 7))
plt.plot(train_ts, label='Training Data')
plt.plot(test_ts, label='Actual Test Data')
plt.plot(test_ts.index, auto_arima_forecast, label='Auto ARIMA Forecast', color='green')
plt.title(f'Auto ARIMA{best_order} Forecast')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.tight_layout()
plt.savefig('plot_10_auto_arima_forecast.png')
plt.show()


**Interpretation:**


Auto ARIMA automatically searches for the optimal ARIMA model parameters (p, d, q) by:
1. Testing different combinations of parameters
2. Evaluating models using information criteria (AIC/BIC)
3. Selecting the model with the lowest information criterion

The best model order is determined based on this search process, which saves time compared to manual testing of multiple models.

Note: Auto ARIMA may sometimes select a different model than what domain knowledge would suggest, so it's important to evaluate the results critically.


## 4. ARIMAX and SARIMAX Models

ARIMAX Model concept not found

SARIMAX Model concept not found

In [ ]:

# Create an exogenous variable (e.g., trading volume)
# For demonstration, we'll use the trading volume as an exogenous variable
exog_train = np.log(df['Volume'][:train_size]).values.reshape(-1, 1)
exog_test = np.log(df['Volume'][train_size:]).values.reshape(-1, 1)

# Fit SARIMAX model with exogenous variable
# Using SARIMAX(1,1,1)(0,0,0,0) which is equivalent to ARIMAX(1,1,1)
sarimax_model = SARIMAX(
    train_ts,
    exog=exog_train,
    order=(1, 1, 1),
    seasonal_order=(0, 0, 0, 0)  # No seasonality
)
sarimax_results = sarimax_model.fit(disp=False)

# Print model summary
print("SARIMAX(1,1,1) Model Summary:")
print(sarimax_results.summary())

# Forecast on the test set
sarimax_forecast = sarimax_results.forecast(steps=len(test_ts), exog=exog_test)

# Calculate error metrics
sarimax_rmse = math.sqrt(mean_squared_error(test_ts, sarimax_forecast))
sarimax_mae = mean_absolute_error(test_ts, sarimax_forecast)
print(f"SARIMAX RMSE: {sarimax_rmse:.4f}")
print(f"SARIMAX MAE: {sarimax_mae:.4f}")

# Plot the forecast
plt.figure(figsize=(14, 7))
plt.plot(train_ts, label='Training Data')
plt.plot(test_ts, label='Actual Test Data')
plt.plot(test_ts.index, sarimax_forecast, label='SARIMAX Forecast', color='purple')
plt.title('SARIMAX(1,1,1) Forecast with Volume as Exogenous Variable')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.tight_layout()
plt.savefig('plot_11_sarimax_forecast.png')
plt.show()


**Interpretation:**


The SARIMAX model extends ARIMA by incorporating:
1. Exogenous variables (X): External factors that may influence the time series (in this case, trading volume)
2. Seasonal components (S): Patterns that repeat at regular intervals

In this example:
- We used log-transformed trading volume as an exogenous variable
- We didn't include seasonality (seasonal_order=(0,0,0,0)) as stock prices typically don't have fixed seasonality

The model summary shows:
- Coefficients for AR, MA, and exogenous terms
- Statistical significance of each parameter
- AIC, BIC, and other model fit statistics

Including the trading volume as an exogenous variable can potentially improve the forecast if volume has predictive power for price movements.


## 5. ARCH and GARCH Models

ARCH/GARCH Model concept not found

In [ ]:

# Fit GARCH(1,1) model on the training log returns
garch_model = arch_model(train_returns, vol='Garch', p=1, q=1)
garch_results = garch_model.fit(disp='off')

# Print model summary
print("GARCH(1,1) Model Summary:")
print(garch_results.summary())

# Forecast volatility
garch_forecast = garch_results.forecast(horizon=len(test_returns))
forecast_vol = np.sqrt(garch_forecast.variance.values[-1, :])

# Plot the volatility forecast
plt.figure(figsize=(14, 7))
plt.plot(train_returns.index[-252:], train_returns[-252:], label='Training Returns', alpha=0.5)
plt.plot(test_returns.index, test_returns, label='Test Returns', alpha=0.5)
plt.plot(test_returns.index, forecast_vol, label='GARCH Volatility Forecast', color='red')
plt.title('GARCH(1,1) Volatility Forecast')
plt.xlabel('Date')
plt.ylabel('Log Returns / Volatility')
plt.legend()
plt.tight_layout()
plt.savefig('plot_12_garch_volatility.png')
plt.show()

# Calculate the 95% confidence intervals for returns
conf_intervals = pd.DataFrame(index=test_returns.index)
conf_intervals['upper'] = 1.96 * forecast_vol
conf_intervals['lower'] = -1.96 * forecast_vol

# Plot returns with confidence intervals
plt.figure(figsize=(14, 7))
plt.plot(test_returns, label='Actual Returns', alpha=0.5)
plt.fill_between(test_returns.index, 
                 conf_intervals['lower'], 
                 conf_intervals['upper'], 
                 color='red', alpha=0.2, 
                 label='95% Confidence Interval')
plt.title('Returns with GARCH 95% Confidence Intervals')
plt.xlabel('Date')
plt.ylabel('Log Returns')
plt.legend()
plt.tight_layout()
plt.show()


**Interpretation:**


GARCH models are specifically designed to model volatility clustering in financial time series:

1. Unlike ARIMA models that forecast the mean (price levels), GARCH models forecast variance (volatility)
2. GARCH(1,1) means the model uses 1 lag of squared returns and 1 lag of past variance
3. The model captures how volatility tends to cluster (periods of high volatility tend to be followed by high volatility)

The model summary shows:
- Coefficients for the constant term (omega), ARCH term (alpha), and GARCH term (beta)
- Statistical significance of each parameter
- Information criteria and other model fit statistics

Applications of GARCH models:
- Risk management and VaR (Value at Risk) calculations
- Option pricing
- Portfolio optimization
- Trading strategies based on volatility forecasts

The 95% confidence intervals show the expected range of returns based on the forecasted volatility.


## 6. Diagnostic Checks and Model Selection

Diagnostic Check concept not found

AIC/BIC concept not found

In [ ]:

# Let's perform diagnostic checks on the Auto ARIMA model residuals
residuals = auto_arima_model.resid()

# Plot residuals
plt.figure(figsize=(14, 7))
plt.subplot(211)
plt.plot(residuals)
plt.title('Residuals of Auto ARIMA Model')
plt.xlabel('Observation')
plt.ylabel('Residual')
plt.subplot(212)
plt.hist(residuals, bins=30)
plt.title('Histogram of Residuals')
plt.xlabel('Residual')
plt.ylabel('Frequency')
plt.tight_layout()
plt.savefig('plot_13_auto_arima_residuals.png')
plt.show()

# Check for autocorrelation in residuals
plt.figure(figsize=(14, 10))
plt.subplot(211)
plot_acf(residuals, lags=40, alpha=0.05, title='ACF of Residuals')
plt.subplot(212)
plot_pacf(residuals, lags=40, alpha=0.05, title='PACF of Residuals')
plt.tight_layout()
plt.savefig('plot_14_residual_acf_pacf.png')
plt.show()

# Ljung-Box test for autocorrelation
lb_test = acorr_ljungbox(residuals, lags=[10, 20, 30], return_df=True)
print("Ljung-Box Test for Autocorrelation in Residuals:")
print(lb_test)

# Compare information criteria for model selection
print("\nInformation Criteria for Model Selection:")
print(f"ARIMA(1,1,1) - AIC: {arima_results.aic:.2f}, BIC: {arima_results.bic:.2f}")
print(f"Auto ARIMA{best_order} - AIC: {auto_arima_model.aic():.2f}, BIC: {auto_arima_model.bic():.2f}")
print(f"SARIMAX(1,1,1) - AIC: {sarimax_results.aic:.2f}, BIC: {sarimax_results.bic:.2f}")


**Interpretation:**


Diagnostic checks are crucial to validate model assumptions and ensure the model is appropriate:

1. Residual Analysis:
   - Residuals should be random with no pattern (white noise)
   - The histogram should approximate a normal distribution
   - No significant autocorrelation should be present in residuals

2. Ljung-Box Test:
   - Tests the null hypothesis that residuals are independently distributed
   - p-value > 0.05 suggests no significant autocorrelation (good)
   - p-value ≤ 0.05 suggests residuals still contain autocorrelation (problematic)

3. Information Criteria:
   - AIC (Akaike Information Criterion) and BIC (Bayesian Information Criterion) balance model fit and complexity
   - Lower values indicate better models
   - BIC penalizes model complexity more heavily than AIC
   - Models with the lowest AIC/BIC are generally preferred

If diagnostic checks reveal issues (e.g., autocorrelated residuals), the model specification should be reconsidered.


## 7. Model Performance Comparison

Model Comparison concept not found

In [ ]:

# Compare model performance on the test set
models = ['ARIMA(1,1,1)', f'Auto ARIMA{best_order}', 'SARIMAX(1,1,1)']
rmse_values = [arima_rmse, auto_arima_rmse, sarimax_rmse]
mae_values = [arima_mae, auto_arima_mae, sarimax_mae]

# Create a comparison dataframe
comparison_df = pd.DataFrame({
    'Model': models,
    'RMSE': rmse_values,
    'MAE': mae_values
})
print("Model Performance Comparison (Test Set):")
print(comparison_df)

# Plot the forecasts from all models
plt.figure(figsize=(14, 7))
plt.plot(train_ts, label='Training Data', alpha=0.3)
plt.plot(test_ts, label='Actual Test Data')
plt.plot(test_ts.index, arima_forecast, label='ARIMA Forecast')
plt.plot(test_ts.index, auto_arima_forecast, label='Auto ARIMA Forecast')
plt.plot(test_ts.index, sarimax_forecast, label='SARIMAX Forecast')
plt.title('Forecast Comparison of Different Models')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.tight_layout()
plt.show()

# Identify the best performing model
best_model_idx = rmse_values.index(min(rmse_values))
print(f"\nBest performing model based on RMSE: {models[best_model_idx]}")
print(f"RMSE: {rmse_values[best_model_idx]:.4f}, MAE: {mae_values[best_model_idx]:.4f}")


**Interpretation:**


Model comparison helps identify the most suitable model for the specific time series:

1. Performance Metrics:
   - RMSE (Root Mean Squared Error): Emphasizes larger errors
   - MAE (Mean Absolute Error): Treats all errors equally
   - Lower values indicate better performance

2. Visual Comparison:
   - Comparing forecasts visually can reveal patterns that metrics might miss
   - Some models might perform better during specific market conditions

3. Model Selection Considerations:
   - Performance metrics (RMSE, MAE)
   - Model complexity and interpretability
   - Information criteria (AIC, BIC)
   - Computational efficiency
   - Domain knowledge and specific requirements

4. Trade-offs:
   - Simpler models (e.g., ARIMA) may be more interpretable but less accurate
   - Complex models may fit better but risk overfitting
   - Including exogenous variables (SARIMAX) can improve performance if the variables have predictive power

The best model depends on the specific forecasting goals and constraints.


## Overall Findings & Recommendations


Based on our analysis of time series models for stock price forecasting:

1. Model Selection:
   - ARIMA models provide a solid baseline for time series forecasting
   - Auto ARIMA can efficiently identify appropriate model orders
   - SARIMAX can incorporate external factors that may influence stock prices
   - GARCH models are valuable for volatility forecasting and risk assessment

2. Key Insights:
   - Stock prices typically require differencing to achieve stationarity
   - Low-order ARIMA models often perform well for stock price forecasting
   - Trading volume can provide additional information as an exogenous variable
   - Volatility clustering is effectively captured by GARCH models

3. Practical Recommendations:
   - Start with simple models (e.g., ARIMA) as a baseline
   - Use diagnostic checks to validate model assumptions
   - Consider domain knowledge when selecting exogenous variables
   - Combine mean models (ARIMA/SARIMAX) with volatility models (GARCH) for comprehensive analysis
   - Regularly retrain models as new data becomes available

4. Limitations:
   - Time series models primarily capture patterns in historical data
   - Unexpected events and market shocks are difficult to predict
   - Stock prices are influenced by many factors beyond historical patterns
   - Model performance can vary across different market conditions

5. Next Steps:
   - Explore machine learning approaches (Class 3)
   - Consider ensemble methods combining multiple models
   - Incorporate additional exogenous variables (economic indicators, sentiment data)
   - Evaluate models across different time horizons and market conditions
